In [51]:
try:
	import google.colab
	!pip install --upgrade pip https://github.com/AlexandreFoley/QuantiT/releases/download/v0.1.1/quantit-0.1.1-cp37-cp37m-linux_x86_64.whl
except:
	pass

In [ ]:
import quantit as qtt
import torch
no_gpu_warning = torch.rand([],device = torch.device('cuda'))

# QuantiT

QuantiT is an open-source library that implement tensors with conservation laws and selection rules and algorithms to manipulate such tensors. Such tensors are commonly used in tensor networks computation to exploit symmetry of a problem to increase efficiency and accuracy of a simulation.

QuantiT is implemented in C++ and offers a python interface as well. Because its tensor implementation is backed by pytorch's tensors, computation can easily done on any backend supported by pytorch, such as GPU and CPU.

## Why Tensor Networks?

Tensor networks allow us to work with a compressed representation of wavefunction and operators. A well designed tensor network allow us to work and optimize the compressed representation without doing a complete decompression. It has proven successful so far: there are many commonly used networks that are applied to problem for which storing a complete representation of a wavefunction would require (several orders of magnitude) more memory than there is on earth.


Let me first explain the basic of tensors and the graphical notation and then explore two of the simplest tensor networks: the matrix product state and matrix product operator.

### Tensors and the graphical notation

Tensor are objects that exist in vectors spaces. Tensors can be represented by multidimensionnal table of coefficients. Vector and Matrices are exemple of 1 dimensionnal and 2 dimensionnal tensors, respectively.
Tensors can be drawn as boxes with as many legs as it has dimensions.

<img src="tikzfigs/Figs-figure0.svg?1" alt="Vector" height="75"> &emsp; &emsp; &emsp;
<img src="tikzfigs/Figs-figure1.svg?1" alt="Matrice" height="75"> &emsp; &emsp; &emsp;
<img src="tikzfigs/Figs-figure2.svg?1" alt="Rank 3 tensor" height="150">

The matrix vector product can be drawn like so:

<img src="tikzfigs/Figs-figure3.svg?1" alt="produit" height="75"> &emsp; &emsp; &emsp;

The shared leg between the two tensors signify a summed over index. In standard equation form, this would be $$ \sum_{ij} M_{ij} V_j |i\rangle $$ where $|i\rangle$ is the working basis set.

Higher rank tensors can be constructed from the contraction of several lower rank tensors. Consider the following exemple:

<img src="tikzfigs/Figs-figure4.svg?1" alt="état" height="150">

The network on the left has $2d\chi + 2d\chi^2$ coefficients while the left has $d^4$ where $d$ is the dimension of index labeled $i,j,k,l$ and $\chi$ the dimensions of the contracted indices. In essence, the network on the left is a parametrization with a polynomial number of coefficient of a wave function in a Hilbert space of size $d^4$.


## Fermions and tensor networks

The Pauli exclusion principle make numerical representation of fermionic quantity a bit trickier than bosons.
Pauli's principle translate mathematically into anticommutation relations for the annihilation and creation operators.
In turn, this create two requirement for a numerical representation: a conventionnal ordering of space and a non-local action for the operators.

When using tensor network, we need perfectly local representation for all our operators. Therefore, we must break apart the creation and annihilation operator into a combination of local components.

Let's consider the spin-half case, we have the annihilation and creation operators $c_{i\sigma}$ and $c_{i\sigma}^\dagger$ where $i$ is the position index, and $\sigma$ the spin index.
The anticommutation relations are $\{c^\dagger_{i\sigma},c_{j\sigma'}\} = \delta_{ij}\delta_{\sigma\sigma'}$ and $\{c_{i\sigma},c_{j\sigma'}\} = 0$

We break the operator appart like so:

$ c_{i\sigma} = (\prod_{j=0}^{i-1} F_j)\tilde{c}_{i\sigma}$ 


where both $F_j$ and $\tilde{c}_{i\sigma}$ are completly local operator, meaning that they **commute** with any local operator that has a different position in space. 
In order to maintain the canonical anticomutation relations, we find that $\tilde{c}_{j\sigma}$ must anticommute with $F_j$, $\tilde{c}_{j\sigma}^2 = 0$, $F_i^2 = I$ and $\{\tilde{c}^\dagger_{j\sigma},\tilde{c}_{j\sigma'}\} = \delta_{\sigma\sigma'}$

Putting all that together, we find that the solution for spin half fermions is this:

In [4]:
c_up,c_dn,F,id = qtt.operators.fermions()
c_dag_up = c_up.conj().permute([1,0])
c_dag_dn = c_dn.conj().permute([1,0])
print("c_up\n",c_up)
print("c_dn\n",c_dn)
print("F\n",F)



c_up
 tensor([[0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]], dtype=torch.int8)
c_dn
 tensor([[ 0,  0,  1,  0],
        [ 0,  0,  0, -1],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]], dtype=torch.int8)
F
 tensor([[ 1,  0,  0,  0],
        [ 0, -1,  0,  0],
        [ 0,  0, -1,  0],
        [ 0,  0,  0,  1]], dtype=torch.int8)


With those constructs, we find that the fermion numbers operator are unchanged by the decomposition, and that hopping terms have a string of F operator between the 2 sites of the hop:
$c_{i\sigma}^\dagger c_{j\sigma} = \tilde{c}_{i\sigma}^\dagger \left(\prod_{k=\mathrm{min}(i,j)}^{k < \mathrm{max}(i,j)}F_k\right) \tilde{c}_{j\sigma}$.

First neighbor hops are then $\tilde{c}_{i\sigma}^\dagger F_i \tilde{c}_{i+1,\sigma}$

## Symmetries and conservations laws

Conservations laws of systems can be taken into account explicitly by tensor network methods. Doing so require enforcing and tracking the conserved quantities inside each tensor that make up a network.

For abelian conserved quantities, such as particle number or z-axis spin, doing so lead to a (hyper-) block structure in the tensor. We proceed by assigning a conseverd quantity to each possible index of each dimension of the tensor and an overall selection rule must be assigned to the tensor as a whole. Let's consider exemples with spin half fermions:


 &emsp; <img src="tikzfigs/Figs-figure6.svg?1" alt="état" height="250">
<img src="tikzfigs/Figs-figure7.svg?1" alt="état" height="290">

The number inside each block is the selection rule that allow non-zero element inside the block.
The particle number assigned to bras has to be the opposite of the kets.
Each dimensions of the tensor have 3 sections: one of size 1 for the empty state, one of size 2 for the single particle states and one of size 2 for the 2 particles state.

Quantit allow you to create the local fermions operators by specifying the conserved value you want to associate with each of their index. In the following exemple with assign

In [5]:
pnum = qtt.conserved.Z;
HS = qtt.btensor([[[1,pnum(0)],[2,pnum(1)],[1,pnum(2)]]],pnum(0))
print(HS)

c_up,c_dn,F,id = qtt.operators.fermions(qtt.shape_from([HS,HS.conj()]))
c_dag_up = c_up.conj().permute([1,0])
c_dag_dn = c_dn.conj().permute([1,0])
print("c_up\n",c_up)
print("c_dn\n",c_dn)
print("F\n",F)

btensor rank 1
 selection rule [Z(0)]
 number of sections by dim [3]
 sections sizes [1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)]]

c_up
 btensor rank 2
 selection rule [Z(-1)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [0, 1]
  1  0
[ CPUCharType(1,2) ]
block at [1, 2]
  0
 1
[ CPUCharType(2,1) ]

c_dn
 btensor rank 2
 selection rule [Z(-1)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [0, 1]
  0  1
[ CPUCharType(1,2) ]
block at [1, 2]
 -1
 0
[ CPUCharType(2,1) ]

F
 btensor rank 2
 selection rule [Z(0)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [0, 0]
  1
[ CPUCharType(1,1) ]
block at [1, 1]
 -1  0
 0 -1
[ CPUCharType(2,2) ]
blo

## Building a MPO

Let's put this all together to build a MPO. The usual strategy when manually building a MPO is to view the rank 4 tensors that make it up as a matrix of quantum operator, and put element in it such that doing a matrix product of multiple such matrix accumulate the Hamilton in the lower left corner.

Within this framework, the a translation invariant 1D Hubbard hamiltonian can be build from the following matrix of operators:

$$
H = \sum_{i \sigma}\left( -t (c^\dagger_{i\sigma} c_{i+1,\sigma} + c^\dagger_{i+1,\sigma}c_{i\sigma}) -\mu n_{i\sigma} + \frac{U}{2} n_{i\uparrow}n_{i\downarrow} \right)
$$

$$
\newcommand{\Id}{\unicode{x1d7d9}}
T_i = \begin{array}{ c c c c c c}
	I & 0 & 0 & 0 & 0 & 0 \\
	\tilde{c}_{i\uparrow} & 0 & 0 & 0 & 0  & 0 \\
	\tilde{c}_{i\downarrow} & 0 & 0 & 0 & 0  & 0 \\
	\tilde{c}_{i\uparrow}^\dagger & 0 & 0 & 0 & 0  & 0 \\
	\tilde{c}_{i\downarrow}^\dagger & 0 & 0 & 0 & 0  & 0 \\
	U n_{i\uparrow} n_{i\downarrow} - \mu (n_{i\uparrow} + n_{i\downarrow}) 
	& t  F_i \tilde{c}^\dagger_{i\uparrow} & t F_i \tilde{c}_{i\downarrow}^\dagger  
	& t \tilde{c}_{i\uparrow} F_i & t \tilde{c}_{i\downarrow} F_i & I \\
\end{array}
$$

and the lower left corner of $T_i T_{i+1}$ is $ \sum_{j = i}^{i+1} \left( U n_{j\uparrow} n_{j\downarrow} - \mu (n_{j\uparrow} + n_{j\downarrow})\right) + \sum_\sigma -t \left( \tilde{c}_{i\sigma}^\dagger F_i \tilde{c}_{i+1,\sigma}  + \tilde{c}_{i+1\sigma}^\dagger F_{i} \tilde{c}_{i,\sigma}\right)$

We will now construct this tensor using quantit's tools.



To do so we must first identify the conserved quantity to assign to each index. A conserving Hamiltonian always leave a conserved quantity unchanged, its overall selection rule must be 0 in this case. What goes on the bonds is the inverse of the selection rule of the operator we have put there. 
The row must then have [0,1,1,-1,-1,0] and the columns [0,-1,-1,1,1,0]

In [6]:
# constant values for the Hubbard hamiltonian
U = qtt.full([],pnum(0),6)
mu = qtt.full([],pnum(0),3)
t = qtt.full([],pnum(0),1)

leftbond = qtt.btensor([[[1,pnum(0)],[2,pnum(1)],[2,pnum(-1)],[1,pnum(0)]]],pnum(0))
rightbond = leftbond.conj()

T = qtt.shape_from([leftbond,HS,rightbond,HS.conj()])

n_up = c_dag_up.bmm(c_up)
n_dn = c_dag_dn.bmm(c_dn)


H_l = -mu * (n_up + n_dn) + U*(n_up.bmm(n_dn))

T.basic_index_put_([0,-1,0,-1], id)
T.basic_index_put_([1,-1,0,-1], c_up)
T.basic_index_put_([2,-1,0,-1], c_dn)
T.basic_index_put_([3,-1,0,-1], c_dag_up)
T.basic_index_put_([4,-1,0,-1], c_dag_dn)
T.basic_index_put_([5,-1,0,-1], H_l)
T.basic_index_put_([5,-1,1,-1], t*F.bmm(c_dag_up))
T.basic_index_put_([5,-1,2,-1], t*F.bmm(c_dag_dn))
T.basic_index_put_([5,-1,3,-1], t*qtt.bmm(c_up,F))
T.basic_index_put_([5,-1,4,-1], t*qtt.bmm(c_dn,F))
T.basic_index_put_([5,-1,5,-1], id)

print(T)

btensor rank 4
 selection rule [Z(0)]
 number of sections by dim [6, 3, 6, 3]
 sections sizes [1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(1)], [Z(-1)], [Z(-1)], [Z(0)], [Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-1)], [Z(1)], [Z(1)], [Z(0)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [0, 0, 0, 0]
 (1,1,.,.) = 
  1
[ CPUFloatType(1,1,1,1) ]
block at [0, 1, 0, 1]
 (1,1,.,.) = 
  1  0

(1,2,.,.) = 
  0  1
[ CPUFloatType(1,2,1,2) ]
block at [0, 2, 0, 2]
 (1,1,.,.) = 
  1
[ CPUFloatType(1,1,1,1) ]
block at [1, 0, 0, 1]
 (1,1,.,.) = 
  1  0
[ CPUFloatType(1,1,1,2) ]
block at [1, 1, 0, 2]
 (1,1,.,.) = 
  0

(1,2,.,.) = 
  1
[ CPUFloatType(1,2,1,1) ]
block at [2, 0, 0, 1]
 (1,1,.,.) = 
  0  1
[ CPUFloatType(1,1,1,2) ]
block at [2, 1, 0, 2]
 (1,1,.,.) = 
 -1

(1,2,.,.) = 
  0
[ CPUFloatType(1,2,1,1) ]
block at [3, 1, 0, 0]
 (1,1,.,.) = 
  1

(1,2,.,.) = 
  0
[ CPUFloatType(1,2,1,1) ]
block at [3, 2, 0, 1]
 (1,1,.,.) = 
  0  1
[ CPUFloatType(1

## Density Matrix Renormalization Group (DMRG)

DMRG is perhaps the simplest tensor network method. In the tensor network formalism, it optimize a matrix product state such that it minimizes the energy of a hamiltonian encoded as a MPO.

This method accomplishes its objective by optimizing the MPS one tensor at a time using a clever trick. The introduction presence of bonds in the MPS reprensentation of the state give use a lot of extra gauge freedom, by choosing the "cannonical" gauge, the optimal solution to any single tensors is the solution of an eigenvalue problem.

The canonical gauge is such that all the MPS's tensors but one are isometric. The one tensor that isn't isometric is called the orthogonality center of the MPS. In the following sketch of a canonical MPS

<img src="tikzfigs/Figs-figure8.svg?1" alt="état" height="90">

The orthogonality center is located at site 2, the tensors labeled $A$ are left isometric and $B$ are right isometric. This mean that the following contraction trivially reduce to the identity:

<img src="tikzfigs/Figs-figure9.svg?1" alt="état" height="180"> &emsp; <img src="tikzfigs/Figs-figure10.svg?1" alt="état" height="180">

The minimization problem at the orthognality center then become quite simple:

 <img src="tikzfigs/Figs-figure11.svg?1" alt="état" height="270">

The solution of this minimization problem is an eigenvalue problem.

DMRG use this identity to efficiently update the orthogonality center of the MPS, and moves the orthogonality center from site to site in order to optimize every tensors of the MPS.

 

In [49]:
H = qtt.networks.bMPO(20,T)
H[0] = H[0].basic_create_view([5,-1,-1,-1],preserve_rank=True)
H[19] = H[19].basic_create_view([-1,-1,0,-1],preserve_rank=True)
H.coalesce()

opt = qtt.algorithms.dmrg_options(convergence_criterion=1e-3)
opt?
E,psi = qtt.algorithms.dmrg(H,pnum(20),opt)


Type:           dmrg_options
String form:    <quantit.quantit.algorithms.dmrg_options object at 0x7f6eb9ac86f0>
Docstring:      <no docstring>
Init docstring: __init__(self: quantit.quantit.algorithms.dmrg_options, *, cutoff: float = 1e-06, convergence_criterion: float = 1e-05, max_bond: int = 18446744073709551615, min_bond: int = 4, maximum_iterations: int = 1000, state_gradient: bool = False, hamil_gradient: bool = False) -> None


In [47]:
for a in psi:
	print(a.sizes())

print("E = ", E.item())

[1, 4, 4]
[4, 4, 16]
[16, 4, 62]
[62, 4, 137]
[137, 4, 186]
[186, 4, 209]
[209, 4, 222]
[222, 4, 236]
[236, 4, 236]
[236, 4, 247]
[247, 4, 244]
[244, 4, 238]
[238, 4, 233]
[233, 4, 213]
[213, 4, 191]
[191, 4, 139]
[139, 4, 63]
[63, 4, 16]
[16, 4, 4]
[4, 4, 1]
-68.13492584228516


# And now, on the GPU!

Caution mac user, you likely don't have a cuda device

In [10]:
H_gpu = H.to(torch.float32,device = torch.device('cuda'))

In [48]:
E_gpu,psi_gpu = qtt.algorithms.dmrg(H,pnum(20),opt)
for a in psi_gpu:
	print(a.sizes())

print("E = ", E_gpu.item())

[1, 4, 4]
[4, 4, 16]
[16, 4, 61]
[61, 4, 141]
[141, 4, 193]
[193, 4, 219]
[219, 4, 234]
[234, 4, 249]
[249, 4, 247]
[247, 4, 253]
[253, 4, 242]
[242, 4, 245]
[245, 4, 233]
[233, 4, 218]
[218, 4, 191]
[191, 4, 141]
[141, 4, 62]
[62, 4, 16]
[16, 4, 4]
[4, 4, 1]
E =  -68.13592529296875


## Computation of average values.

Quantit offer tools to compute the average value of MPO. The function *contract* from the *networks* submodule can compute expectation values of MPO with MPS.


In [50]:
avg_H = qtt.networks.contract(psi,psi,H)
print("<H> = ",avg_H.item())

<H> =  -68.13516998291016


We can take advantage of the canonical reprensentation of the MPS to compute local observable faster. Let's consider the following exemple: the contribution of (any) two sites to the kinetic energy.

We first build the two site MPO for it.

In [46]:

kin_bond = qtt.btensor([[[2,pnum(1)],[2,pnum(-1)]]],pnum(0))
Kin_2s_left = qtt.shape_from([HS,kin_bond,HS.conj()])
Kin_2s_right = qtt.shape_from([kin_bond.conj(),HS,HS.conj()])
Kin_2s_left.basic_index_put_([-1,0,-1],c_up)
Kin_2s_left.basic_index_put_([-1,1,-1],c_dn)
Kin_2s_left.basic_index_put_([-1,2,-1],c_dag_up)
Kin_2s_left.basic_index_put_([-1,3,-1],c_dag_dn)
Kin_2s_right.basic_index_put_([0,-1,-1],t*F.bmm(c_dag_up))
Kin_2s_right.basic_index_put_([1,-1,-1],t*F.bmm(c_dag_dn))
Kin_2s_right.basic_index_put_([2,-1,-1],t*qtt.bmm(c_up,F))
Kin_2s_right.basic_index_put_([3,-1,-1],t*qtt.bmm(c_dn,F))

print()

We then move the orthogonality center of the ground state to the target site, site 9 for this exemple.

In [52]:
psi.move_oc(9)
print(psi.orthogonality_center)

9


We can then extract a two site MPS from psi, and compute its kinetic energy

In [45]:



K_89 = qtt.tensordot(psi[8],Kin_2s_left,dims=([1],[2]))
K_89 = qtt.tensordot(K_89,psi[8].conj(),dims=([0,2],[0,1]))
K_89 = qtt.tensordot(K_89,psi[9],dims=([0],[0]))
K_89 = qtt.tensordot(K_89,Kin_2s_right,dims=([0,2],[0,2]))
K_89 = qtt.tensordot(K_89,psi[9].conj(),dims=([0,1,2],[0,2,1]))

print(K_89.item())


0.7346468567848206
